Obtain the Washington Geological Survey Landslide data from the official website: https://dnr.wa.gov/washington-geological-survey/geologic-hazards-and-environment/landslides

Create a directory 'Data' in your projects root folder and place the downloaded files inside it.

Also create a directory 'ProcessedDataSets' inside of your PreProcessing Directory

# Initialization

In [1]:
import fiona
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

gdb_dir = "../Data/Washington/ger_portal_landslides_ds29_v1.4_Apr2024/WGS_Landslides.gdb"

layers = fiona.listlayers(gdb_dir)
print("Layers in geodatabase:")
for lyr in layers:
    print("  -", lyr)


Layers in geodatabase:
  - scarp_and_flank
  - rock_fall_deposit
  - rock_fall_scarp
  - recent_landslide_point
  - SLIP_fans
  - landslide_compilation_study_footprints
  - study_areas
  - SLIP_landslide
  - fan
  - landslide_deposit
  - landslide_compilation
  - scarp


# Initial Inspection

In [2]:
deposits = gpd.read_file(gdb_dir, layer="landslide_deposit")

/opt/anaconda3/envs/nlp/lib/python3.13/site-packages/pyogrio/raw.py:198: RuntimeWarning: Non closed ring detected. To avoid accepting it, set the OGR_GEOMETRY_ACCEPT_UNCLOSED_RING configuration option to NO
  return ogr_read(


In [3]:
# Basic info
print(deposits.shape)      # (n_deposits, n_fields)
print(deposits.dtypes)     # field names and types

(15060, 23)
LANDSLIDE_ID                        int32
MATERIAL                           object
MOVEMENT                           object
MOVE_CODE                          object
CONFIDENCE                         object
RELATIVE_AGE                       object
YEAR_MOVE                          object
FIELD_VERIFIED                     object
SLOPE_DEG                           int16
HS_HEIGHT_FT                      float64
FAIL_DEPTH_FT                     float64
MVMT_AZIMUTH_DEG                  float32
VOLUME_FT3                        float64
AVG_SCARP_DIST_FT                 float64
LS_NAME                            object
COMMENTS                           object
DISPLAY_ON_PORTAL                  object
CHANGE_TYPE                        object
CHANGE_DATE           datetime64[ms, UTC]
CHANGE_DESCRIPTION                 object
Shape_Length                      float64
Shape_Area                        float64
geometry                         geometry
dtype: object


# New DataFrame Creation

In [4]:
columns_to_keep = [
    'LANDSLIDE_ID', 
    'MATERIAL', 
    'MOVEMENT', 
    'MOVE_CODE',
    'CONFIDENCE',
    'RELATIVE_AGE', 
    'YEAR_MOVE',
    'SLOPE_DEG', 
    'HS_HEIGHT_FT', 
    'FAIL_DEPTH_FT',
    'MVMT_AZIMUTH_DEG',
    'VOLUME_FT3',
    'AVG_SCARP_DIST_FT', 
    'LS_NAME',
    'Shape_Length',
    'Shape_Area',
    'geometry'
]

# Create a new GeoDataFrame with only the selected columns
washington_landslides = gpd.GeoDataFrame(deposits[columns_to_keep], geometry='geometry', crs=deposits.crs)

## Inspect the new DataFrame

In [5]:
print("New GeoDataFrame shape:", washington_landslides.shape)
print("\nColumn names:")
for col in washington_landslides.columns:
    print(f"  - {col}")

washington_landslides.head()

New GeoDataFrame shape: (15060, 17)

Column names:
  - LANDSLIDE_ID
  - MATERIAL
  - MOVEMENT
  - MOVE_CODE
  - CONFIDENCE
  - RELATIVE_AGE
  - YEAR_MOVE
  - SLOPE_DEG
  - HS_HEIGHT_FT
  - FAIL_DEPTH_FT
  - MVMT_AZIMUTH_DEG
  - VOLUME_FT3
  - AVG_SCARP_DIST_FT
  - LS_NAME
  - Shape_Length
  - Shape_Area
  - geometry


,LANDSLIDE_ID,MATERIAL,MOVEMENT,MOVE_CODE,CONFIDENCE,RELATIVE_AGE,YEAR_MOVE,SLOPE_DEG,HS_HEIGHT_FT,FAIL_DEPTH_FT,MVMT_AZIMUTH_DEG,VOLUME_FT3,AVG_SCARP_DIST_FT,LS_NAME,Shape_Length,Shape_Area,geometry
0,18414,Earth or debris,Flow,EFL,Low (1-10),Pre-historic (>150 years),None,11,3.0,3.0,225.0,1322.894339,NaN,None,82.334854,440.966914,"MULTIPOLYGON (((1043516.638 614643.978, 104351..."
1,20629,Earth or debris,Complex,ES-R+EFL,Low (1-10),Pre-historic (>150 years),None,39,11.0,9.0,180.0,4039.516598,NaN,None,77.534531,441.695598,MULTIPOLYGON (EMPTY)
2,20630,Earth or debris,Complex,ES-R+EFL,Low (1-10),Pre-historic (>150 years),None,37,11.0,9.0,180.0,4563.138478,NaN,None,81.931760,483.669276,MULTIPOLYGON (EMPTY)
3,20661,Earth or debris,Topple,ET,Moderate (11-29),Pre-historic (>150 years),None,61,24.0,12.0,225.0,6320.755277,NaN,None,85.493104,509.669212,"MULTIPOLYGON (((1032543.812 660981.578, 103254..."
4,20594,Earth or debris,Topple,ET,Moderate (11-29),Pre-historic (>150 years),None,33,29.0,24.0,135.0,13016.550042,NaN,None,91.365031,528.125198,MULTIPOLYGON (EMPTY)


### Save the DataFrame

In [6]:
washington_landslides.to_file("./ProcessedDataSets/washington_landslides_processed.geojson", driver="GeoJSON")

In [7]:
loaded_landslides = gpd.read_file("./ProcessedDataSets/washington_landslides_processed.geojson")
print("Loaded GeoDataFrame shape:", loaded_landslides.shape)
loaded_landslides.head()

Loaded GeoDataFrame shape: (15060, 17)


,LANDSLIDE_ID,MATERIAL,MOVEMENT,MOVE_CODE,CONFIDENCE,RELATIVE_AGE,YEAR_MOVE,SLOPE_DEG,HS_HEIGHT_FT,FAIL_DEPTH_FT,MVMT_AZIMUTH_DEG,VOLUME_FT3,AVG_SCARP_DIST_FT,LS_NAME,Shape_Length,Shape_Area,geometry
0,18414,Earth or debris,Flow,EFL,Low (1-10),Pre-historic (>150 years),None,11,3.0,3.0,225.0,1322.894339,NaN,None,82.334854,440.966914,"MULTIPOLYGON (((1043516.638 614643.978, 104351..."
1,20629,Earth or debris,Complex,ES-R+EFL,Low (1-10),Pre-historic (>150 years),None,39,11.0,9.0,180.0,4039.516598,NaN,None,77.534531,441.695598,MULTIPOLYGON (EMPTY)
2,20630,Earth or debris,Complex,ES-R+EFL,Low (1-10),Pre-historic (>150 years),None,37,11.0,9.0,180.0,4563.138478,NaN,None,81.931760,483.669276,MULTIPOLYGON (EMPTY)
3,20661,Earth or debris,Topple,ET,Moderate (11-29),Pre-historic (>150 years),None,61,24.0,12.0,225.0,6320.755277,NaN,None,85.493104,509.669212,"MULTIPOLYGON (((1032543.812 660981.578, 103254..."
4,20594,Earth or debris,Topple,ET,Moderate (11-29),Pre-historic (>150 years),None,33,29.0,24.0,135.0,13016.550042,NaN,None,91.365031,528.125198,MULTIPOLYGON (EMPTY)
